In [1]:
import sys
sys.path.append('../')

import time
import warnings
import pandas as pd
import xgboost as xgb
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from utils.features.feature_extraction import Feature_extraction


from utils.constant import FEATURES, LABELS, SYN_FLOOD_ATTACKS

In [2]:
start = time.time()
pcapfiles = [
    '../inputs/sniffed.pcap',
]
destination_directory = '../outputs/'
n_threads = 8

address = "./"

for i in range(len(pcapfiles)):
    lstart = time.time()
    pcap_file = pcapfiles[i]
    fe = Feature_extraction()
    df = fe.pcap_evaluation(pcap_file,destination_directory + pcap_file)
    print(f'done! ({pcap_file})(' + str(round(time.time()-lstart, 2))+ 's)')
    
end = time.time()
print(f'Elapsed Time = {(end-start)}s')
df

done! (../inputs/sniffed.pcap)(0.21s)
Elapsed Time = 0.2075648307800293s


,ts,Protocol_name,Duration,Protocol Type,flow_duration,Header_Length,src_ip_bytes,fin_flag_number,syn_flag_number,rst_flag_number,...,Weight,Correlation,RARP,DS status,Fragments,Sequence number,Protocol Version,flow_idle_time,flow_active_time,source_ip
0,1.701046e+09,UDP,128,17,0.000000,99,1,0,0,0,...,1,0.0,0,0,0,0,0,1.701046e+09,0.000000,192.168.43.110
1,1.701046e+09,UDP,128,17,0.000000,99,2,0,0,0,...,4,NaN,0,0,0,0,0,1.701046e+09,0.000000,192.168.43.110
2,1.701046e+09,UDP,64,17,0.051106,294,1,0,0,0,...,9,1.0,0,0,0,0,0,5.110598e-02,0.051106,192.168.43.65
3,1.701046e+09,UDP,64,17,0.063181,306,2,0,0,0,...,16,1.0,0,0,0,0,0,6.318092e-02,0.063181,192.168.43.65
4,1.701046e+09,TCP,128,6,0.000000,66,3,0,1,0,...,25,1.0,0,0,0,0,0,1.701046e+09,0.000000,192.168.43.110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,1.701046e+09,UDP,128,17,0.000000,75,49,0,0,0,...,36,1.0,0,0,0,0,0,1.701046e+09,0.000000,192.168.43.110
206,1.701046e+09,UDP,64,17,0.057788,200,5,0,0,0,...,49,1.0,0,0,0,0,0,5.778790e-02,0.057788,192.168.43.65
207,1.701046e+09,UDP,64,17,0.057927,166,6,0,0,0,...,64,1.0,0,0,0,0,0,5.792689e-02,0.057927,192.168.43.65
208,1.701046e+09,UDP,64,17,0.058138,178,7,0,0,0,...,81,1.0,0,0,0,0,0,5.813789e-02,0.058138,192.168.43.65


In [3]:
# Preprocessing Functions
scaler = StandardScaler()
scaler.fit(df[FEATURES])
df[FEATURES] = scaler.transform(df[FEATURES])

df[df['source_ip'] == '192.168.43.221'][FEATURES]

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight
68,-0.389951,-0.544157,-0.765628,-0.297601,2.264905,2.264905,0.0,-0.139347,-0.139347,-0.199007,...,-1.097635,-0.189836,0.316542,-0.235109,-0.054057,-1.340122,-0.190112,-0.290722,0.768706,-0.462908
69,-0.389951,-0.544157,-0.765628,-0.297601,2.264905,2.264905,0.0,-0.139347,-0.139347,-0.199007,...,-0.941537,-0.087587,0.316542,-0.235109,0.120590,-1.066104,-0.087920,-0.268643,0.768706,-0.304842
70,-0.389951,-0.544157,-0.765628,-0.297601,2.264905,2.264905,0.0,-0.139347,-0.139347,-0.199007,...,-0.813820,-0.033319,0.316542,-0.235109,0.295237,-0.865027,-0.033682,-0.254842,0.768706,-0.130137
71,-0.389951,-0.544157,-0.765628,-0.297601,2.264905,2.264905,0.0,-0.139347,-0.139347,-0.199007,...,-0.707390,-0.002184,0.316542,-0.235109,0.469884,-0.709500,-0.002565,-0.246271,0.768706,0.061206
72,-0.389951,-0.544157,-0.765628,-0.297601,2.264905,2.264905,0.0,-0.139347,-0.139347,-0.199007,...,-0.617333,0.015812,0.316542,-0.235109,0.644531,-0.584929,0.015422,-0.241100,0.768706,0.269188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,-0.389951,-0.544157,-0.765628,-0.297601,-0.410038,-0.410038,0.0,-0.139347,-0.139347,-0.199007,...,0.172421,-0.142275,0.316542,-0.235109,1.343120,0.327181,-0.142577,-0.281090,0.768706,1.267499
177,-0.389951,-0.544157,-0.765628,-0.297601,-0.410038,-0.410038,0.0,-0.139347,-0.139347,-0.199007,...,0.188584,-0.152075,0.316542,-0.235109,1.517767,0.343444,-0.152372,-0.283166,0.768706,1.558674
178,-0.389951,-0.544157,-0.765628,-0.297601,-0.410038,-0.410038,0.0,-0.139347,-0.139347,-0.199007,...,0.203045,-0.161292,0.316542,-0.235109,1.692414,0.357933,-0.161584,-0.285075,0.768706,1.866487
179,-0.389951,-0.544157,-0.765628,-0.297601,-0.410038,-0.410038,0.0,-0.139347,-0.139347,-0.199007,...,0.216060,-0.169975,0.316542,4.253341,-1.625882,0.357933,-0.161584,-0.285075,0.768706,1.866487


# Predict with our model

In [4]:
xgb_model = xgb.XGBClassifier()
xgb_model.load_model('../models/xgb.model.bin.json')

In [5]:
preds = xgb_model.predict(df[FEATURES])
sum(preds)


185